[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cyberviser/Hancock/blob/main/Hancock_Universal_Finetune.ipynb)

# 🔐 Hancock Universal Fine-Tuning — CyberViser
**Mistral 7B → Cybersecurity specialist via LoRA**

Works on: **Google Colab** (free T4) · **Kaggle** (free T4, 30h/week) · **RunPod/Vast** · **Oracle Cloud**

| Step | Time | Notes |
|------|------|-------|
| Install deps | ~3 min | Unsloth + TRL |
| Load 4-bit model | ~2 min | 7B params, 4GB VRAM |
| Train (300 steps) | ~25 min | v3 dataset (5,670 samples) |
| Export GGUF Q4 | ~5 min | Ready for Ollama |

**Setup:**
- **Colab:** Runtime → Change runtime type → T4 GPU
- **Kaggle:** Settings → Accelerator → GPU T4 x2 · Internet → On
- **Optional:** Add `HF_TOKEN` secret to push model to HuggingFace Hub

In [ ]:
# @title 1️⃣  Detect Environment
import os, platform

ENV = 'Unknown'
if os.path.exists('/kaggle'):
    ENV = 'Kaggle'
    WORK_DIR = '/kaggle/working'
elif 'COLAB_GPU' in os.environ or os.path.exists('/content'):
    ENV = 'Colab'
    WORK_DIR = '/content'
else:
    ENV = 'Cloud/Local'
    WORK_DIR = os.getcwd()

print(f'💻 Environment: {ENV}')
print(f'📁 Work dir: {WORK_DIR}')
os.chdir(WORK_DIR)

In [ ]:
# @title 2️⃣  Install Dependencies (~3 min)
import subprocess, sys

# Unsloth install varies by environment
if ENV == 'Kaggle':
    !pip install 'unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git' -q
else:
    !pip install 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git' -q

!pip install -q 'trl>=0.8.6' 'transformers>=4.40' 'datasets>=2.18' peft huggingface_hub accelerate bitsandbytes

import importlib, pkg_resources
for pkg in ['unsloth','trl','transformers','datasets','peft']:
    v = pkg_resources.get_distribution(pkg).version
    print(f'  {pkg}: {v}')
print('\n✅ Dependencies installed')

In [ ]:
# @title 3️⃣  Clone Hancock & Check GPU
import torch

# Clone repo
if not os.path.exists(os.path.join(WORK_DIR, 'Hancock')):
    !git clone https://github.com/cyberviser/Hancock.git {WORK_DIR}/Hancock
os.chdir(os.path.join(WORK_DIR, 'Hancock'))

# GPU check
assert torch.cuda.is_available(), '❌ Enable GPU runtime first!'
gpu = torch.cuda.get_device_name(0)
vram = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f'GPU: {gpu} | VRAM: {vram:.1f} GB')
print(f'Repo: {os.getcwd()}')
print('✅ Ready')

In [ ]:
# @title 4️⃣  Configure Training
# Adjust these settings before running:

TRAINING_STEPS = 300        # More steps = better quality, longer time (~5 min per 100 steps on T4)
EXPORT_GGUF = True          # Export GGUF for Ollama deployment
PUSH_TO_HUB = False         # Push to HuggingFace Hub (requires HF_TOKEN)
LORA_RANK = 0               # 0 = auto-detect based on VRAM (16=T4, 32=24GB, 64=40GB+)

print(f'Steps: {TRAINING_STEPS}')
print(f'GGUF export: {EXPORT_GGUF}')
print(f'Push to Hub: {PUSH_TO_HUB}')
print(f'LoRA rank: {"auto" if LORA_RANK == 0 else LORA_RANK}')

In [ ]:
# @title 5️⃣  Run Fine-Tuning (~25 min on T4)
# Uses the universal hancock_finetune_v3.py script

cmd = f'python hancock_finetune_v3.py --steps {TRAINING_STEPS}'
if EXPORT_GGUF:
    cmd += ' --export-gguf'
if PUSH_TO_HUB:
    cmd += ' --push-to-hub'
if LORA_RANK > 0:
    cmd += f' --lora-r {LORA_RANK}'

print(f'Running: {cmd}\n')
!{cmd}

In [ ]:
# @title 6️⃣  Test the Fine-Tuned Model
import torch
from unsloth import FastLanguageModel
from pathlib import Path

adapter_dir = 'hancock-adapter-v3'
assert Path(adapter_dir).exists(), f'Adapter not found at {adapter_dir}'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=adapter_dir,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

messages = [
    {'role': 'system', 'content': 'You are Hancock, an elite cybersecurity AI by CyberViser.'},
    {'role': 'user', 'content': 'Explain CVE-2021-44228 Log4Shell and provide a Splunk SPL query to detect exploitation attempts.'},
]
inputs = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors='pt'
).to('cuda')

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs, max_new_tokens=512,
        use_cache=True, temperature=0.7, do_sample=True,
    )

response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print('\n🛡️ Hancock says:')
print('=' * 60)
print(response)

In [ ]:
# @title 7️⃣  Download Model Files
import os
from pathlib import Path

print('\n📦 Available model files:\n')

# LoRA adapter
adapter_dir = Path('hancock-adapter-v3')
if adapter_dir.exists():
    size = sum(f.stat().st_size for f in adapter_dir.rglob('*') if f.is_file()) / 1e6
    print(f'  📁 hancock-adapter-v3/ ({size:.0f} MB) — LoRA adapter')

# GGUF
for gguf in Path('.').glob('*.gguf'):
    size = gguf.stat().st_size / 1e6
    print(f'  📁 {gguf.name} ({size:.0f} MB) — GGUF for Ollama')

print('\n⬇️  Download options:')
if ENV == 'Colab':
    print('  Option 1: Run the cell below to download via browser')
    print('  Option 2: Mount Google Drive and copy there')
elif ENV == 'Kaggle':
    print('  Option 1: Output tab → download files from /kaggle/working/Hancock/')
    print('  Option 2: Push to HuggingFace Hub (re-run with PUSH_TO_HUB=True)')
else:
    print('  Files are saved in the current directory')

In [ ]:
# @title 8️⃣  Download GGUF (Colab only)
# Skip this cell on Kaggle — use Output tab instead
import os

try:
    from google.colab import files
    for f in os.listdir('.'):
        if f.endswith('.gguf'):
            print(f'Downloading {f}...')
            files.download(f)
            break
    else:
        print('No GGUF file found. Run with EXPORT_GGUF=True')
except ImportError:
    print('Not running on Colab — download manually from the Output tab (Kaggle) or local filesystem')

---
## 🚀 Deploy with Ollama (after download)

```bash
# On your local machine:
mkdir -p ~/cyberviser/models && mv hancock-v3-Q4_K_M.gguf ~/cyberviser/models/
cd ~/cyberviser/Hancock

# Update Modelfile to point to GGUF:
# FROM ./models/hancock-v3-Q4_K_M.gguf

ollama create hancock-finetuned -f Modelfile.hancock-finetuned
ollama run hancock-finetuned
```

---
© 2026 CyberViser · [cyberviser.netlify.app](https://cyberviser.netlify.app)